# Tarea 2: Question Answering Fine-tuning

In [3]:
# Librerías

# !pip install -U datasets fsspec
# !pip install evaluate

import warnings
warnings.filterwarnings('ignore')

import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

import torch
print("Is CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Number of GPUs available:", torch.cuda.device_count())

from time import time
from datasets import *
from transformers import *
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.colheader_justify', 'center')

Is CUDA available: True
CUDA version: 12.4
Number of GPUs available: 1


## Dataset

El dataset de SQuAD (Stanford Question Answering Dataset) es un conjunto de datos utilizado principalmente para entrenar y evaluar modelos de comprensión lectora. Consiste en ternas de preguntas, respuestas y contexto.

Aquí la ficha del dataset para que podáis explorarla: https://huggingface.co/datasets/rajpurkar/squad

In [2]:
# No modificar esta celda
# Esta celda, celda tiene que estar ejecutada en la entrega

dataset = load_dataset("squad")
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

Con el único motivo de no demorar los tiempos de entrenamiento. Filtraremos el dataset y nos quedaremos solo con los registros que tenga longitud del campo _context_ inferior a 300.

El resto de la práctica se pide trabajarla sobre la variable `ds_tarea`.

In [3]:
# No modificar esta celda
# Esta celda, celda tiene que estar ejecutada en la entrega

def filtra_por_longitud(ejemplo):
    return len(ejemplo["context"]) < 300

ds_tarea = dataset.filter(filtra_por_longitud)

assert len(ds_tarea['train']) == 3466
assert len(ds_tarea['validation']) == 345

ds_tarea

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 3466
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 345
    })
})

## Modeling

En este apartado es donde tendréis que realizar todo el trabajo de la práctica. El formato, el análisis, el modelo escogido y cualquier proceso intermedio que consideréis es totalmente libre. Sin embargo, hay algunas pautas que tendréis que cumplir:

- La variable `model_checkpoint` debe almacenar el nombre del modelo y el tokenizador de 🤗 que vais a utilizar.
- La variable `model` y la variable `tokenizer` almacenarán, respectivamente, el modelo y el tokenizador de 🤗 que vais a utilizar.
- La variable `trainer` almacenará el _Trainer_ de 🤗 que, en la siguiente sección utilizaréis para entrenar el modelo.

In [4]:
model_checkpoint = 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
def preprocess_function(examples):
    inputs = tokenizer(
        examples['question'],
        examples['context'],
        truncation = 'only_second',
        max_length = 384,
        stride = 128,
        return_overflowing_tokens = True,
        return_offsets_mapping = True,
        padding = 'max_length'
    )

    sample_mapping = inputs.pop("overflow_to_sample_mapping")
    offset_mapping = inputs.pop("offset_mapping")

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = inputs['input_ids'][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = inputs.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples['answers'][sample_index]

        if len(answers['answer_start']) == 0:
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            start_char = answers['answer_start'][0]
            end_char = start_char + len(answers['text'][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                start_positions.append(cls_index)
                end_positions.append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                start_position = token_start_index - 1

                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                end_position = token_end_index + 1

                start_positions.append(start_position)
                end_positions.append(end_position)

    inputs['start_positions'] = start_positions
    inputs['end_positions'] = end_positions
    return inputs

ds_tokenized = ds_tarea.map(preprocess_function, batched = True, remove_columns = ds_tarea['train'].column_names)

data_collator = DefaultDataCollator()

training_args = TrainingArguments(
    output_dir = './qa_model',
    eval_strategy = 'epoch',
    save_strategy = 'epoch',
    learning_rate = 2e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 3,
    weight_decay = 0.01,
    logging_dir = './logs',
    report_to = 'none',
    seed = 42
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = ds_tokenized['train'],
    eval_dataset = ds_tokenized['validation'],
    tokenizer = tokenizer,
    data_collator = data_collator
)

loading configuration file config.json from cache at C:\Users\jorge.gperea\.cache\huggingface\hub\models--roberta-base\snapshots\e2da8e2f811d1448a5b465c236feacd80ffbac7b\config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.41.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at C:\Users\jorge.gperea\.cache\huggingface\hub\models--roberta-base\snapshots\e2da8e2f811d1448a5b465c236feacd8

## Training

In [8]:
# No modificar esta celda
# Esta celda, celda tiene que estar ejecutada en la entrega

start = time()

trainer.train()

end = time()
print(f">>>>>>>>>>>>> elapsed time: {(end-start)/60:.0f}m")

{'eval_loss': 1.3043107986450195, 'eval_runtime': 7.0839, 'eval_samples_per_second': 48.702, 'eval_steps_per_second': 6.211, 'epoch': 1.0}
{'loss': 1.6188, 'grad_norm': 41.177574157714844, 'learning_rate': 1.2334869431643626e-05, 'epoch': 1.1520737327188941}
{'eval_loss': 1.2893723249435425, 'eval_runtime': 7.0738, 'eval_samples_per_second': 48.771, 'eval_steps_per_second': 6.22, 'epoch': 2.0}
{'loss': 0.7743, 'grad_norm': 42.40425109863281, 'learning_rate': 4.654377880184332e-06, 'epoch': 2.3041474654377883}
{'eval_loss': 1.355137586593628, 'eval_runtime': 7.0854, 'eval_samples_per_second': 48.692, 'eval_steps_per_second': 6.21, 'epoch': 3.0}
{'train_runtime': 952.5473, 'train_samples_per_second': 10.916, 'train_steps_per_second': 1.367, 'train_loss': 1.0547791699293754, 'epoch': 3.0}
>>>>>>>>>>>>> elapsed time: 16m


## Evaluation

In [9]:
# No modificar esta celda
# Esta celda, celda tiene que estar ejecutada en la entrega

print(f"**** EVALUACIÓN ****")
print(f"********\nTokenizer config:\n{tokenizer}")
print(f"\n\n********\nModel config:\n{model.config}")
print(f"\n\n********\nTrainer arguments:\n{trainer.args}")

**** EVALUACIÓN ****
********
Tokenizer config:
RobertaTokenizerFast(name_or_path='roberta-base', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}
)


********
Model config:
RobertaConfig {
  "architectu

In [10]:
# No modificar esta celda
# Esta celda, celda tiene que estar ejecutada en la entrega

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

question_answerer = pipeline("question-answering", model=model, tokenizer=tokenizer, device=device)

In [11]:
# No modificar esta celda
# Esta celda, celda tiene que estar ejecutada en la entrega

assert len(ds_tarea['train']) == 3466
assert len(ds_tarea['validation']) == 345

def calculate_sentence_similarity(sentence1, sentence2):
    sentence1 = re.sub(r'[^a-zA-Z0-9\s]', '', sentence1).lower()
    sentence2 = re.sub(r'[^a-zA-Z0-9\s]', '', sentence2).lower()
    words1 = set(sentence1.lower().split())
    words2 = set(sentence2.lower().split())
    matches = len(words1.intersection(words2))
    total_words = len(words1.union(words2))
    if total_words == 0:
        return 0.0
    return (matches / total_words) * 100

samples = [324,342,249,176,70,168,120,58,90,192,278,289,197,146,323,248,260,273,112,211]
evaluation_list = []

for ii in samples:
    context = ds_tarea['validation'][ii]['context']
    question = ds_tarea['validation'][ii]['question']
    answer = ds_tarea['validation'][ii]['answers']
    answers = [f"{tt}" for ii, tt in enumerate(answer['text'])]
    prediction = question_answerer(context=context, question=question)['answer']
    match = max([calculate_sentence_similarity(w, prediction) for w in answers])
    evaluation_list.append((ii,context,question,answers,prediction,match))

print(f"*** evaluation_df ***")
evaluation_df = pd.DataFrame(evaluation_list, columns=['sample', 'context', 'question', 'real_answers', 'predicted_answer', 'match'])
evaluation_df[['sample','real_answers','predicted_answer', 'match']]

*** evaluation_df ***


,sample,real_answers,predicted_answer,match
0,324,"[Hospitality Business/Financial Centre, Downtown Riverside, Hospitality Business/Financial Centre]",Hospitality Business/Financial Centre,100.000000
1,342,"[Rugby, Rugby, Rugby]",Rugby,100.000000
2,249,"[extremely high, high, extremely high]",extremely high,100.000000
3,176,"[""A Machine to End War"", ""A Machine to End War"", A Machine to End War]",A Machine to End War,100.000000
4,70,"[Death Wish Coffee, Death Wish Coffee, Death Wish Coffee]",Death Wish Coffee,100.000000
5,168,"[antagonistic, antagonistic, antagonistic]",antagonistic,100.000000
6,120,"[1892 to 1894, from 1892 to 1894, from 1892 to 1894]",1892 to 1894,100.000000
7,58,"[Vince Lombardi Trophy, the Vince Lombardi Trophy, Vince Lombardi Trophy]",Vince Lombardi Trophy,100.000000
8,90,"[5 Live Sports Extra, 5 Live Sports Extra, 5 Live Sports Extra]",5 Live Sports Extra,100.000000
9,192,"[time, time complexity, time complexity]","best, worst and average case complexity",14.285714


### Criterio de evaluación

La **nota final de la tarea2** estará relacionada con el resultado de las predicciones de vuestro modelo.

El criterio de evaluación será el siguiente:

- La tarea2 se aprobará si el notebook se entrega sin fallos y con un modelo entrenado (independientemente de sus predicciones).
- Se ponderará en función de la columna _match_, que otorga 100% de acierto si todas las palabras coinciden y bajará gradualmente el porcentaje de acierto en función del número de palabras que no coincidan.
    
Nota: La nota que se calcula a continuación es orientativa y podría verse reducida en función del código de la entrega.

In [12]:
print(f"Tu nota de la tarea2 es: {max(np.ceil(evaluation_df['match'].sum() / len(evaluation_df) / 10), 5.0)}")

Tu nota de la tarea2 es: 9.0
